In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from bokeh.io import show, output_notebook

from bokeh.plotting import figure

import seaborn as sns
import matplotlib.pylab as pyplot

In [2]:
def get_world_population(table_title, url):
    """
    Obtener una tabla con el nombre especificado en la URL pasada y devolverla como dataframe
    """
    ########### Su código
    try:
        pd.read_html(url, match=table_title, attrs={"class": "wikitable sortable"}) # "wikitable sortable"(tablas)
    finally:
        title = table_title + "*"
        
        return pd.read_html(url, match=title, attrs={"class": "wikitable sortable"})[0].set_index('Region')

In [3]:
def get_estimations_json(population_table, dest_file):
    """
    Recibe un dataframe con la población, devolder un json solo con los años estimados (los posteriores al actual)
    """
    ########### Su código

    current_year = 2019 # extraer año por funciones
    return population_table.loc[:, population_table.columns.astype('int64') > current_year].to_json(dest_file)

In [4]:
def get_bokeh(df):
    """
    Obtener un simple gráfico con la librería bokeh comparando la evolución de la población por continentes
    """
    
    colors = [
        "coral", 
        "darkblue", 
        "dimgray", 
        "fuchsia", 
        "gold", 
        "red", 
        "yellow",        
        
    ]
    
    ########### Su código
    from bokeh.plotting import figure, show
    from bokeh.io import output_notebook
    #from bokeh.palettes import Paired7

    p = figure(x_axis_label='Year', y_axis_label='Population') #plot_width=800, plot_height=250, x_axis_type="datetime"

    Region = df.index.to_list()
    L = len(Region)

    for region, color in zip(Region, colors):
        p.line(df.columns, df.loc[region, "1500":], line_width=2, color=color, legend_label=region)

    p.legend.location = "top_left"
    p.title.text = 'World historical and predicted populations'
    
    output_notebook()
    
    show(p)

In [130]:
def get_seaborn(df):
    """
    Obtener un simple gráfico con la librería seaborn comparando la evolución de la población por continentes
    """
    ########### Su código
    df = df.reset_index().melt('Region', var_name = 'Year', value_name = 'Population')
    
    sns.set(rc={'figure.figsize':(10,10)})
    
    sns.lineplot(x='Year', y='Population', hue='Region', data=df)

In [8]:
def get_excel(df, nombre):
    """
    Volcar el dataframe pasado a una hoja de cálculo con los siguientes coloreando de:
        verde: Población por debajo de 100 millones de personas.
        rojo: Poblaciones por encima de 100 millones de personas.
        
    Estos colores son solo aplicables a continentes, no al total del mundo.
    """
    ###########Your code

    def highlight_100M(df):
        if df < 100:
            color = 'green'
        else:
            color = 'red'
        
        return 'color: %s' % color

    return df.style.applymap(highlight_100M, subset=pd.IndexSlice[df.index[1:], :]).to_excel(nombre)